In [ ]:
import sys
import os

from pathlib import Path
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

global PATH
PATH = "results/*"


os.chdir("C:/Users/rcwyuen/OneDrive/Studies/UCL/Publications/TCR-Embeddings")
sys.path.append("C:/Users/rcwyuen/OneDrive/Studies/UCL/Publications/TCR-Embeddings")


def plot_auc_result(data, category, execution, method):
    best_aucs = []
    for kfold, results in data.items():
        plotting = []
        for train_record in results[category]:
            df = pd.read_csv(Path.cwd() / execution / kfold / train_record)
            plotting.append(
                (
                    int(str(train_record.parent).replace("Epoch ", "")),
                    roc_auc_score(df["actual"], df["pred"]),
                )
            )

        plotting.sort(key=lambda x: -x[1])
        best_aucs.append(plotting[0][1])
    return method(best_aucs)


def load_results(method):
    executions = set((Path.cwd().glob(PATH)))  # l2-normed/
    executions = executions - {
        Path.cwd() / "results/complete",
        Path.cwd() / "results/l2-normed",
        Path.cwd() / "results/l2-not-normed",
    }

    df = {}

    for execution in tqdm(list(executions)):
        data = {}
        for folder in list(execution.glob("kfold-*")):
            data[str(folder.relative_to(execution))] = {
                "train": [
                    i.relative_to(execution / folder)
                    for i in folder.glob("Epoch */train-records.csv")
                ],
                "test": [
                    i.relative_to(execution / folder)
                    for i in folder.glob("Epoch */test-records.csv")
                ],
                # "eval": [i.relative_to(execution / folder) for i in folder.glob("Epoch */eval-records.csv")],
            }
        try:
            df[str(execution.relative_to(Path.cwd() / "results"))] = {
                "train": plot_auc_result(data, "train", execution, method),
                # "eval": plot_auc_result(data, "eval", execution, method),
                "test": plot_auc_result(data, "test", execution, method),
            }
        except IndexError:
            df[str(execution.relative_to(Path.cwd() / "results"))] = {
                "train": None,
                "test": None,
                # "eval": "no-data",
            }

    return pd.DataFrame(df).T.sort_index()

In [ ]:
df = load_results(np.mean).sort_values("test")
df

In [28]:
def encoding_method(x):
    if "johnson-lindenstarauss" in x:
        return "johnson-lindenstarauss"
    elif "no-reduction" in x:
        return "no-reduction"
    else:
        return "autoencoder"


df_ = df.copy()
df_["Reduction Method"] = df_.reset_index()["index"].apply(encoding_method).tolist()
df_["Embedding Method"] = (
    df_.reset_index()["index"]
    .apply(lambda x: x.replace(f"-{encoding_method(x)}", ""))
    .tolist()
)
df_ = df_.reset_index().drop("index", axis=1)
df_["K-Fold Train AUC"] = df_["train"].apply(lambda x: round(x, 5))
df_["K-Fold Test AUC"] = df_["test"].apply(lambda x: round(x, 5))

df_[
    ["Reduction Method", "Embedding Method", "K-Fold Train AUC", "K-Fold Test AUC"]
].to_csv("stats.csv", index=False)

Accuracy

In [ ]:
def plot_auc_result(data, category, execution, method):
    best_aucs = []
    for kfold, results in data.items():
        plotting = []
        for train_record in results[category]:
            df = pd.read_csv(Path.cwd() / execution / kfold / train_record)
            plotting.append(
                (
                    int(str(train_record.parent).replace("Epoch ", "")),
                    np.mean((df["pred"] > 0.5).astype(int) == df["actual"]),
                )
            )

        plotting.sort(key=lambda x: -x[1])
        best_aucs.append(plotting[0][1])
    return method(best_aucs)


def load_results(method):
    executions = set((Path.cwd().glob(PATH)))  # l2-normed/
    executions = executions - {
        Path.cwd() / "results/complete",
        Path.cwd() / "results/l2-normed",
        Path.cwd() / "results/l2-not-normed",
    }

    df = {}

    for execution in tqdm(list(executions)):
        data = {}
        for folder in list(execution.glob("kfold-*")):
            data[str(folder.relative_to(execution))] = {
                "train": [
                    i.relative_to(execution / folder)
                    for i in folder.glob("Epoch */train-records.csv")
                ],
                "test": [
                    i.relative_to(execution / folder)
                    for i in folder.glob("Epoch */test-records.csv")
                ],
                # "eval": [i.relative_to(execution / folder) for i in folder.glob("Epoch */eval-records.csv")],
            }
        try:
            df[str(execution.relative_to(Path.cwd() / "results"))] = {
                "train": plot_auc_result(data, "train", execution, method),
                # "eval": plot_auc_result(data, "eval", execution, method),
                "test": plot_auc_result(data, "test", execution, method),
            }
        except IndexError:
            df[str(execution.relative_to(Path.cwd() / "results"))] = {
                "train": None,
                "test": None,
                # "eval": "no-data",
            }

    return pd.DataFrame(df).T.sort_index()


df = load_results(np.mean).sort_values("test")
df